Крок 1: Імпорт бібліотек

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, make_scorer, r2_score, mean_absolute_error



Крок 2: Завантаження даних

In [ ]:
train_data = pd.read_csv('../datasets/mod_04_hw_train_data.csv')
valid_data = pd.read_csv('../datasets/mod_04_hw_valid_data.csv')

Крок 3: Первинний дослідницький аналіз (EDA)


In [ ]:
# Перегляд структури даних

# Перегляд перших рядків тренувального набору даних
print(train_data.head())

# Отримання інформації про датасет
print(train_data.info())

# Статистичний огляд даних
print(train_data.describe())

# Перевірка на наявність пропущених значень
print(train_data.isnull().sum())

# Візуалізація розподілу змінних
sns.pairplot(train_data)
plt.show()

# Відбір лише числових колонок
numerical_data = train_data.select_dtypes(include=[np.number])

# Кореляційна матриця для числових змінних
sns.heatmap(numerical_data.corr(), annot=True, fmt=".2f")
plt.show()


Крок 4: Підготовка даних

In [ ]:
from scipy import stats

# Виділення числових даних для аналізу
numerical_data = train_data.select_dtypes(include=[np.number])

# Обрахунок Z-оцінки для числових колонок
z_scores = np.abs(stats.zscore(numerical_data))

# Видалення рядків, де будь-яка числова колонка має Z-оцінку більше 3
filtered_data = numerical_data[(z_scores < 3).all(axis=1)]

# Відображення кількості видалених рядків з викидами
print("Видалено рядків з викидами:", numerical_data.shape[0] - filtered_data.shape[0])

# Створення копії датафрейму для безпечної модифікації
filtered_data = filtered_data.copy()

# Заповнення пропущених значень медіаною для кожної колонки
filtered_data = filtered_data.fillna(filtered_data.median())

# Перевірка пропущених значень після обробки
print(filtered_data.isnull().sum())

In [ ]:
# Виділення категоріальних і числових ознак
categorical_features = train_data.select_dtypes(include=['object']).columns.tolist()
numerical_features = train_data.select_dtypes(include=[np.number]).columns.tolist()
numerical_features.remove('Salary')  # Видалення цільової змінної

# Створення пайплайнів для обробки ознак
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Заповнення пропусків медіаною
    ('scaler', StandardScaler())                    # Нормалізація даних
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Заповнення пропусків
    ('onehot', OneHotEncoder(handle_unknown='ignore'))                      # One-hot кодування
])

# Об'єднання трансформерів у колонковий трансформер
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])


In [ ]:
# Визначення нерелевантних ознак
irrelevant_features = ['Date_Of_Birth', 'Qualification', 'Cert']

# Перевірка наявності та видалення нерелевантних ознак
for feature in irrelevant_features:
    if feature in train_data.columns:
        train_data.drop(columns=[feature], inplace=True)
    if feature in valid_data.columns:
        valid_data.drop(columns=[feature], inplace=True)

# Виділення категоріальних і числових ознак
categorical_features = train_data.select_dtypes(include=['object']).columns.tolist()
numerical_features = train_data.select_dtypes(include=[np.number]).columns.tolist()
if 'Salary' in numerical_features:
    numerical_features.remove('Salary')  # Видалення цільової змінної

# Пайплайн для числових ознак
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Пайплайн для категоріальних ознак
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Комбінування обробки ознак
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Створення пайплайну з моделлю
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', KNeighborsRegressor())
])


Крок 5: Пошук оптимальних параметрів за допомогою GridSearchCV

In [ ]:
param_grid = {
    'regressor__n_neighbors': [3, 5, 7, 9, 11, 13, 15, 17, 19, 25, 50],
    'regressor__weights': ['uniform', 'distance'],
    'regressor__p': [1, 2]  # 1 - Мангеттенська відстань, 2 - Евклідова відстань
}

grid_search = GridSearchCV(model_pipeline, param_grid, cv=5, scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False))
grid_search.fit(train_data.drop('Salary', axis=1), train_data['Salary'])

print(f"Найкращі параметри: {grid_search.best_params_}")


Крок 6: Оцінка моделі



In [ ]:
# Використання оптимальної моделі для валідаційних даних
best_model = grid_search.best_estimator_
y_pred = best_model.predict(valid_data.drop('Salary', axis=1))
mape = mean_absolute_percentage_error(valid_data['Salary'], y_pred)
print(f'Validation MAPE: {mape:.2%}')



# Розрахунок метрик
mape = mean_absolute_percentage_error(valid_data['Salary'], y_pred)
mse = mean_squared_error(valid_data['Salary'], y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(valid_data['Salary'], y_pred)
mae = mean_absolute_error(valid_data['Salary'], y_pred)

print(f'MAPE: {mape:.2%}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')
print(f'MAE: {mae}')


In [ ]:
# Аналогічна обробка валідаційного набору
X_valid = valid_data.drop('Salary', axis=1)
y_valid = valid_data['Salary']


ТЕПЕР ДРУГА ЧАСТИНА ЗАВДАННЯ: етапи обробки і підготовки даних для валідаційного набору, подібно до того, як у темі «Метод опорних векторів (SVM)» в розділі «Практика застосування SVM-класифікатора. Навчання й оцінка моделі. Приклад використання моделі

Крок 2: Завантаження даних

In [ ]:
# Завантаження даних
trained_data = pd.read_csv('../datasets/mod_04_hw_valid_data.csv')


Крок 3: Первинний дослідницький аналіз (EDA)

In [ ]:
# Перегляд структури даних
print(trained_data.head())
print(trained_data.describe())
print(trained_data.info())

# Перевірка на відсутні значення
print(trained_data.isnull().sum())


Крок 4: Підготовка даних

In [ ]:
# Визначення нерелевантних ознак
irrelevant_features = ['Date_Of_Birth', 'Qualification', 'Cert']

# Перевірка наявності та видалення нерелевантних ознак
for feature in irrelevant_features:
    if feature in trained_data.columns:
        trained_data.drop(columns=[feature], inplace=True)

# Виділення категоріальних і числових ознак
categorical_features = trained_data.select_dtypes(include=['object']).columns.tolist()
numerical_features = trained_data.select_dtypes(include=[np.number]).columns.tolist()
if 'Salary' in numerical_features:
    numerical_features.remove('Salary')  # Видалення цільової змінної

# Пайплайн для числових ознак
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Пайплайн для категоріальних ознак
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Комбінування обробки ознак
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Створення пайплайну з моделлю
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', KNeighborsRegressor())
])


Крок 5: Пошук оптимальних параметрів за допомогою GridSearchCV

In [ ]:
X1_train = trained_data.drop('Salary', axis=1)
y2_train = trained_data['Salary']

param_grid = {
    'regressor__n_neighbors': [3, 5],
    'regressor__weights': ['uniform', 'distance'],
    'regressor__p': [1, 2]  # 1 - Мангеттенська відстань, 2 - Евклідова відстань
}

grid_search = GridSearchCV(model_pipeline, param_grid, cv=5, scoring=make_scorer(mean_absolute_percentage_error, greater_is_better=False))
grid_search.fit(X1_train, y2_train)

print(f"Найкращі параметри: {grid_search.best_params_}")


Крок 6: Оцінка моделі

Крок 7: Прогноз заробітної плати для нових працівників

In [ ]:
# Створення нового набору даних для прогнозу
new_data = pd.DataFrame(
    data={
        'Name': ['John Doe'],
        'Experience': [5],
        'University': ['University A'],
        'Qualification': ['Masters'],
        'Role': ['Manager'],
        'Location': ['New York'],
        'Phone_Number': ['1234567890'],
        'Date_Of_Birth': ['1980-01-01'],
        'Cert': ['CertA']
    }
)

# Видалення нерелевантних ознак
for feature in irrelevant_features:
    if feature in new_data.columns:
        new_data.drop(columns=[feature], inplace=True)

# Прогноз заробітної плати
new_data_transformed = best_model['preprocessor'].transform(new_data)
salary_pred = best_model['regressor'].predict(new_data_transformed)

print(f'Прогнозована заробітна плата для нового працівника: {salary_pred[0]}')
